In [10]:
#Importamos el conjunto de datos Breat Cancer Wisconsin, con 569 muestras de celulas tumorales benignas y malignas. 
# LAs dos priemras columnas alamcenan lo sId unicos de las muestras y sus daignosticos, M MAligna y B Benigna. Las columnas 3-32 
#continene 30 caracteristicas de valor real actualizadas a partir de imagenes digitalizadas de nucleos celualres. 
#Leemos el conjunto de datos 

import pandas as pd
df=pd.read_csv("wdbc.data",header=None)

In [11]:
#Preparamos X e y
X=df.iloc[:,2:].values
y=df.iloc[:,1].values

#Hacemos LabelEncoder de su representacion M B a enteros
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)
le.classes_

array(['B', 'M'], dtype=object)

In [12]:
#Split the data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=0)

In [13]:
# Aprendemos los pipelines y combinamos transformadores y estimadores
#Vamos a fit and transform un StandardSCaler y sacar lo sPCAs y finalmente aplicar un LogisticREgression

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline 

pipe_lr=make_pipeline(StandardScaler(),
                     PCA(n_components=2),
                     LogisticRegression(random_state=1))
pipe_lr.fit(X_train,y_train)
y_pred=pipe_lr.predict(X_test)
pipe_lr.score(X_test,y_test)


0.9298245614035088

## Validacion cruzada estratificada

In [19]:
# Ahora usamos la validacion cruzada de k iteraciones estratificada. Esta respeta las proporciones de clase
#en cada iteracion para garantizar que cada iteracion representa las proporciones de clase
#en el conjunto de datos.
#FOLLOWING from above

from sklearn.model_selection import StratifiedKFold
import numpy as np

pipe_lr=make_pipeline(StandardScaler(),
                     PCA(n_components=2),
                     LogisticRegression(random_state=1))

kfold=StratifiedKFold(n_splits=10,random_state=None).split(X_train,y_train)
scores=[]

for k,(train,test) in enumerate(kfold):
   # print(train)
    #print(test)
    pipe_lr.fit(X_train[train],y_train[train])
    score=pipe_lr.score(X_train[test],y_train[test])
    scores.append(score)
    print("Fold: %2d, Class dist.: %s, Acc: %.3f" % (k+1,np.bincount(y_train[train]),score))
    

Fold:  1, Class dist.: [256 153], Acc: 0.957
Fold:  2, Class dist.: [256 153], Acc: 0.978
Fold:  3, Class dist.: [256 153], Acc: 0.935
Fold:  4, Class dist.: [256 153], Acc: 0.978
Fold:  5, Class dist.: [256 153], Acc: 1.000
Fold:  6, Class dist.: [257 153], Acc: 0.978
Fold:  7, Class dist.: [257 153], Acc: 0.956
Fold:  8, Class dist.: [257 153], Acc: 0.889
Fold:  9, Class dist.: [257 153], Acc: 0.978
Fold: 10, Class dist.: [257 153], Acc: 1.000


In [15]:
#Repito

kfold=StratifiedKFold(n_splits=10,random_state=None).split(X_train,y_train)
#Me genera 10 matrices de entrenamiento y 10 vectores de test.
scores=[]
#enumerate nos mantiene en el k estamos y nos saca el train y test para ese k.

for k, (train,test) in enumerate(kfold):
   
    pipe_lr.fit(X_train[train],y_train[train])
    score=pipe_lr.score(X_train[test],y_train[test])
    scores.append(score)
    print(k+1,np.bincount(y_train[train]),score)

1 [256 153] 0.9565217391304348
2 [256 153] 0.9782608695652174
3 [256 153] 0.9347826086956522
4 [256 153] 0.9782608695652174
5 [256 153] 1.0
6 [257 153] 0.9777777777777777
7 [257 153] 0.9555555555555556
8 [257 153] 0.8888888888888888
9 [257 153] 0.9777777777777777
10 [257 153] 1.0


In [25]:
#forma smas directa de sklearn
from sklearn.model_selection import cross_val_score

sc=cross_val_score(estimator=pipe_lr,
                   X=X_train,
                  y=y_train,
                  cv=10,
                  n_jobs=1)

print(np.mean(scores))

0.9647826086956522
